In [1]:
import datetime as dt
import os
import pandas as pd
import tweepy as tw

from tweepy.auth import OAuthHandler

In [2]:
# developer permissions
consumerKey = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumerSecret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

auth = OAuthHandler(consumerKey, consumerSecret)

accessToken = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
accessTokenSecret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXx'

auth.set_access_token(accessToken, accessTokenSecret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
#empty lists to recieve json data
name = []
date = []
text = []
like_count = []
rt_count = []


#function to retrieve twitter data
def retrieve_data(handle):
    
    """calls twitter api and returns data and reformats from json into a pandas dataframe"""
    
    try:
        for tweet in tw.Cursor(api.user_timeline,
                               screen_name=handle, 
                               count=200,
                               exclude_replies=True, 
                               include_rts=False,
                               tweet_mode="extended").items(1000):
            
            name.append(tweet.user.screen_name)
            date.append(tweet.created_at)
            text.append(tweet.full_text)
            like_count.append(tweet.favorite_count)
            rt_count.append(tweet.retweet_count)
            
            df = pd.DataFrame()

            df['name'] = name
            df['date'] = date
            df['text'] = text
            df['like_count'] = like_count
            df['rt_count'] = rt_count

    except:
        pass
    return df

In [4]:
#join each individual dataframe into one combined df
df = pd.DataFrame()

handles = ['@JoeBiden', '@ewarren', '@BernieSanders', 
           '@MikeBloomberg', '@PeteButtigieg', '@AmyKlobuchar',
          '@realDonaldTrump']

for handle in handles:
    df_new = retrieve_data(handle)
    df = pd.concat((df, df_new))

In [5]:
# preliminary data cleaning
df.drop_duplicates(keep='first', inplace=True)

df.groupby(['name'])['date'].count()

name
BernieSanders      1000
JoeBiden           1000
MikeBloomberg      1000
PeteButtigieg      1000
amyklobuchar       1000
ewarren            1000
realDonaldTrump    1000
Name: date, dtype: int64

In [6]:
#df.to_csv('candidates_tweets.csv')